In [ ]:
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import joblib

In [ ]:
# Function to fetch news data from News API
def fetch_news(api_key, country='in', category='general', num_articles=100):
    url = f'https://newsapi.org/v2/top-headlines'
    params = {'country': country, 'category': category, 'apiKey': api_key, 'pageSize': num_articles}
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        articles = data.get('articles', [])
        return [article['title'] for article in articles]
    else:
        print(f"Error: {response.status_code}")
        return []

In [ ]:
# Fetch real news using the News API
api_key = '9e65bee657184741a4882c72c6024990'
real_news = fetch_news(api_key, country='in', category='general', num_articles=10)

In [ ]:
real_news

In [ ]:
# Simulate fake news (replace with actual fake news dataset or source)
fake_news = ["Fake news title 1", "Fake news title 2", "Fake news title 3"]

In [ ]:
# Create a DataFrame with labels (1 for real news, 0 for fake news)
df = pd.DataFrame({'title': real_news + fake_news, 'label': [1] * len(real_news) + [0] * len(fake_news)})

In [ ]:
# Shuffle the DataFrame
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['title'], df['label'], test_size=0.2, random_state=42)

In [ ]:
# Define a stemming function
def stemming(text):
    ps = PorterStemmer()
    return ' '.join([ps.stem(word) for word in text.split()])

In [ ]:
# Create a pipeline with TfidfVectorizer, stemming, and Naive Bayes classifier
model = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words=stopwords.words('english'))),
    ('stemming', TfidfVectorizer(analyzer=stemming)),
    ('classifier', MultinomialNB())
])

In [ ]:
# Train the model
model.fit(X_train, y_train)

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

In [ ]:
# Save the trained model and vectorizer
joblib.dump(model, 'model.pkl')
joblib.dump(model.named_steps['vectorizer'], 'vector.pkl')